# Análisis de datos FIFA - Modelo predictivo

## Inicialización

In [1]:
# importar librerias
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
# cargar los datos
fifa = pd.read_csv('https://raw.githubusercontent.com/IreneRA/Hackaton-Tripleten/Raw-data/international_matches.csv')

In [3]:
# vista previa
fifa.head(5)

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#información general
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           23921 non-null  object 
 1   home_team                      23921 non-null  object 
 2   away_team                      23921 non-null  object 
 3   home_team_continent            23921 non-null  object 
 4   away_team_continent            23921 non-null  object 
 5   home_team_fifa_rank            23921 non-null  int64  
 6   away_team_fifa_rank            23921 non-null  int64  
 7   home_team_total_fifa_points    23921 non-null  int64  
 8   away_team_total_fifa_points    23921 non-null  int64  
 9   home_team_score                23921 non-null  int64  
 10  away_team_score                23921 non-null  int64  
 11  tournament                     23921 non-null  object 
 12  city                           23921 non-null 

Descripción de los datos de `fifa`:
-  `date`: fecha en que se llevó a cabo el partido.
-  `home_team`: equipo local o equipo que juega en casa.
-  `away_team`: equipo visitante.
-  `home_team_continent`: continente al que pertenece el equipo local.
-  `away_team_continent`: continente al que pertenece el equipo visitante.
-  `home_team_fifa_rank`: clasificación FIFA del equipo local en el momento del partido.
-  `away_team_fifa_rank`: clasificación FIFA del equipo visitante en el momento del partido.
-  `home_team_total_fifa_points`: puntos totales FIFA del equipo local en el momento del partido.
-  `away_team_total_fifa_points`: puntos totales FIFA del equipo visitante en el momento del partido.
-  `home_team_score`: goles anotados por el equipo local durante el partido.
-  `away_team_score`: goles anotados por el equipo visitante durante el partido.
-  `tournament`: torneo o competición en el que se lleva a cabo el partido.
-  `city`: ciudad en la que se realizó el partido.
-  `country`: país en el que se realizó el partido.
-  `neutral_location`: booleano (True/False) que indica si el partido se llevó a cabo en una ubicación neutral (es decir, no es el hogar de ninguno de los equipos).
-  `shoot_out`: indica si hubo una tanda de penaltis ("Sí" o "No").
-  `home_team_result`: resultado del equipo local (puede ser "Ganar", "Empatar" o "Perder").
-  `home_team_goalkeeper_score`: calificación del portero del equipo local.
-  `away_team_goalkeeper_score`: calificación del portero del equipo visitante.
-  `home_team_mean_defense_score`: calificación media de la defensa del equipo local.
-  `home_team_mean_offense_score`: calificación media del ataque del equipo local.
-  `home_team_mean_midfield_score`: calificación media del mediocampo del equipo local.
-  `away_team_mean_defense_score`: calificación media de la defensa del equipo visitante.
-  `away_team_mean_offense_score`: calificación media del ataque del equipo visitante.
-  `away_team_mean_midfield_score`: calificación media del mediocampo del equipo visitante.

In [5]:
# convertir la columna 'date' a datetime
fifa['date'] = pd.to_datetime(fifa['date'])

# creación nueva columna año
fifa['year'] = fifa['date'].dt.year

Verificar cambios realizados

In [6]:
# información general
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           23921 non-null  datetime64[ns]
 1   home_team                      23921 non-null  object        
 2   away_team                      23921 non-null  object        
 3   home_team_continent            23921 non-null  object        
 4   away_team_continent            23921 non-null  object        
 5   home_team_fifa_rank            23921 non-null  int64         
 6   away_team_fifa_rank            23921 non-null  int64         
 7   home_team_total_fifa_points    23921 non-null  int64         
 8   away_team_total_fifa_points    23921 non-null  int64         
 9   home_team_score                23921 non-null  int64         
 10  away_team_score                23921 non-null  int64         
 11  tournament     

## Modelo predictivo
construir un modelo que prediga el resultado de un partido y compararlo con las predicciones basadas en el ranking FIFA.

Objetivo categórico:
- `home_team_result`: resultado del equipo local (puede ser "Ganar" (1), "Empatar" (0.5) o "Perder" (0)).

Caracéristicas a tomar en cuenta:
- `neutral_location`: booleano (True/False) que indica si el partido se llevó a cabo en una ubicación neutral (es decir, no es el hogar de ninguno de los equipos).
- `home_team_goalkeeper_score`: calificación del portero del equipo local.
- `away_team_goalkeeper_score`: calificación del portero del equipo visitante.
- `home_team_mean_defense_score`: calificación media de la defensa del equipo local.
- `home_team_mean_offense_score`: calificación media del ataque del equipo local.
- `home_team_mean_midfield_score`: calificación media del mediocampo del equipo local.
- `away_team_mean_defense_score`: calificación media de la defensa del equipo visitante.
- `away_team_mean_offense_score`: calificación media del ataque del equipo visitante.
- `away_team_mean_midfield_score`: calificación media del mediocampo del equipo visitante.

In [7]:
#separar variables que se usaran en el modelo
model_df = fifa.loc[:,['home_team_result','neutral_location','home_team_goalkeeper_score','away_team_goalkeeper_score',
                       'home_team_mean_defense_score','home_team_mean_offense_score','home_team_mean_midfield_score',
                       'away_team_mean_defense_score','away_team_mean_offense_score','away_team_mean_midfield_score']]

#quitar filas con valores ausentes
model_df = model_df.dropna()
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4303 entries, 8264 to 23918
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   home_team_result               4303 non-null   object 
 1   neutral_location               4303 non-null   bool   
 2   home_team_goalkeeper_score     4303 non-null   float64
 3   away_team_goalkeeper_score     4303 non-null   float64
 4   home_team_mean_defense_score   4303 non-null   float64
 5   home_team_mean_offense_score   4303 non-null   float64
 6   home_team_mean_midfield_score  4303 non-null   float64
 7   away_team_mean_defense_score   4303 non-null   float64
 8   away_team_mean_offense_score   4303 non-null   float64
 9   away_team_mean_midfield_score  4303 non-null   float64
dtypes: bool(1), float64(8), object(1)
memory usage: 340.4+ KB


Se tienen los datos del tipo correcto y sin valores ausentes

Se segmentan los datos fuente en 3 conjuntos:

- conjunto de entrenamiento (60%)
- conjunto de validación (20%)
- conjunto de prueba (20%)

In [8]:
#Separar conjunto de datos de prueba
df_t, df_test = train_test_split(model_df,test_size=0.2,random_state=12345)

#Separar conjunto de datos de entrenamiento y validacion. El test_size sube para compensar la reducción de filas
df_train, df_valid = train_test_split(df_t,test_size=0.25,random_state=12345)

In [9]:
#Separar variables de conjunto de entrenamiento
df_train_features = df_train.drop('home_team_result',axis=1)
df_train_target =  df_train['home_team_result']

#Separar variables de conjunto de validación
df_valid_features = df_valid.drop('home_team_result',axis=1)
df_valid_target = df_valid['home_team_result']

#Separar variables de conjunto de prueba
df_test_features = df_test.drop('home_team_result',axis=1)
df_test_target = df_test['home_team_result']

#verificar variables y tamaños
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(2581, 10)
(861, 10)
(861, 10)


Se entrenan 3 modelos diferentes y se comparan sus resultados para evaluar su funcionamiento.

###   Modelo: Árbol de decisión para clasificación

In [10]:
#buscar las mejores características con GridSearchCV, para buscar la mejor profundidad de árbol
param_grid = {'max_depth': [1, 2, 3, 4, 5,6,7,8,9,10,20,30,40],'criterion':['gini','entropy'], }

tree_clas = DecisionTreeClassifier(random_state=12345)
grid_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(df_train_features, df_train_target)

final_model = grid_search.best_estimator_
final_model

Fitting 5 folds for each of 26 candidates, totalling 130 fits


DecisionTreeClassifier(max_depth=4, random_state=12345)

In [11]:
#fijar modelo con parámetros encontrados
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=4)
model_dtc.fit(df_train_features, df_train_target)

DecisionTreeClassifier(max_depth=4, random_state=12345)

In [12]:
#predicciones de los 3 conjuntos de datos
train_predictions_dtc = model_dtc.predict(df_train_features)
test_predictions_dtc = model_dtc.predict(df_test_features)
valid_predictions_dtc = model_dtc.predict(df_valid_features)

#comprobar calidad del modelo
print('Accuracy')
print('Training set:',   accuracy_score(df_train_target, train_predictions_dtc))
print('Test set:',       accuracy_score(df_test_target,  test_predictions_dtc))
print('Validation set:', accuracy_score(df_valid_target, valid_predictions_dtc))

Accuracy
Training set: 0.5373886090662534
Test set: 0.5133565621370499
Validation set: 0.5331010452961672


### Modelo: Bosque aleatorio para clasificación

In [13]:
#buscar las mejores características para los datos disponibles
model_rfc =RandomForestClassifier(random_state=12345)
param_grid = {'n_estimators': [1,2,3,4,5,6,7,8,9,10],'max_features': [ 'sqrt', 'log2'],'max_depth' : [3,4,5,6,7,8,9,10]}
CV_rfc = GridSearchCV(estimator=model_rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(df_train_features,df_train_target)
CV_rfc.best_params_

{'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 8}

In [14]:
#fijar modelo con parámetros encontrados
model_rfc = RandomForestClassifier(random_state=12345, max_depth=4, max_features= 'sqrt',n_estimators=8)
model_rfc.fit(df_train_features,df_train_target)

RandomForestClassifier(max_depth=4, n_estimators=8, random_state=12345)

In [15]:
#predicciones de los 3 conjuntos de datos
train_predictions_rfc = model_rfc.predict(df_train_features)
test_predictions_rfc = model_rfc.predict(df_test_features)
valid_predictions_rfc = model_rfc.predict(df_valid_features)

#comprobar calidad del modelo
print('Accuracy')
print('Training set:',   accuracy_score(df_train_target, train_predictions_rfc))
print('Test set:',       accuracy_score(df_test_target,  test_predictions_rfc))
print('Validation set:', accuracy_score(df_valid_target, valid_predictions_rfc))

Accuracy
Training set: 0.5470747772181325
Test set: 0.5087108013937283
Validation set: 0.5412311265969802


### Modelo: Regresión logística

In [16]:
# constructor de regresión logística
model_lr = LogisticRegression(random_state=12345, solver='liblinear')
model_lr.fit(df_train_features,df_train_target)

LogisticRegression(random_state=12345, solver='liblinear')

In [17]:
#predicciones de los 3 conjuntos de datos
train_predictions_lr = model_lr.predict(df_train_features)
test_predictions_lr = model_lr.predict(df_test_features)
valid_predictions_lr = model_lr.predict(df_valid_features)

#comprobar calidad del modelo
print('Accuracy')
print('Training set:',   accuracy_score(df_train_target, train_predictions_lr))
print('Test set:',       accuracy_score(df_test_target,  test_predictions_lr))
print('Validation set:', accuracy_score(df_valid_target, valid_predictions_lr))

Accuracy
Training set: 0.5366137156141031
Test set: 0.5389082462253194
Validation set: 0.537746806039489


Al comparar los 3 modelos, el más confiable, por mantener su exactitud en los 3 grupos es el de regresión logística.

### ¿Que tanta precisión tiene el ranking fifa?

In [18]:
# modelo predictivo segun el ranking FIFA
fifa['rank_predictions'] = np.where(fifa['home_team_fifa_rank'] > fifa['away_team_fifa_rank'],'Win',
                           np.where(abs(fifa['home_team_fifa_rank'] - fifa['away_team_fifa_rank'])<10, 'Draw','Lose'))
fifa.loc[:,['date','home_team','away_team','home_team_result','home_team_fifa_rank','away_team_fifa_rank','rank_predictions']].head(10)

,date,home_team,away_team,home_team_result,home_team_fifa_rank,away_team_fifa_rank,rank_predictions
0,1993-08-08,Bolivia,Uruguay,Win,59,22,Win
1,1993-08-08,Brazil,Mexico,Draw,8,14,Draw
2,1993-08-08,Ecuador,Venezuela,Win,35,94,Lose
3,1993-08-08,Guinea,Sierra Leone,Win,65,86,Lose
4,1993-08-08,Paraguay,Argentina,Lose,67,5,Win
5,1993-08-08,Peru,Colombia,Lose,70,19,Win
6,1993-08-08,Zimbabwe,Eswatini,Win,50,102,Lose
7,1993-08-09,Guinea,Sierra Leone,Win,65,86,Lose
8,1993-08-11,Faroe Islands,Norway,Lose,111,9,Win
9,1993-08-11,Sweden,Switzerland,Lose,4,3,Win


In [19]:
#comprobar calidad del modelo usando ranking fifa
print('Accuracy',   accuracy_score(fifa['home_team_result'], fifa['rank_predictions']))

Accuracy 0.22026671125789055


Historicamente, el ranking FIFA ha tenido cambios por lo que es de esperarse que tenga una baja exactitud.

Se investiga si la exactitud a mejorado recientemente.

In [20]:
#Exactitud tomando los ultimos 10 años (2012 - 2022)
fifa_10 = fifa[fifa['year'] > 2011]
print('Accuracy',   accuracy_score(fifa_10['home_team_result'], fifa_10['rank_predictions']))

Accuracy 0.213089802130898


In [21]:
#Vista previa
fifa_10.loc[:,['date','home_team','away_team','home_team_result','home_team_fifa_rank','away_team_fifa_rank','rank_predictions']].head(10)

,date,home_team,away_team,home_team_result,home_team_fifa_rank,away_team_fifa_rank,rank_predictions
14723,2012-01-06,Equatorial Guinea,South Africa,Draw,150,52,Win
14724,2012-01-07,Botswana,Zimbabwe,Draw,95,98,Draw
14725,2012-01-09,Gabon,Burkina Faso,Draw,77,62,Win
14726,2012-01-09,Tunisia,Sudan,Win,59,113,Lose
14727,2012-01-11,Nigeria,Angola,Draw,43,83,Lose
14728,2012-01-12,Senegal,Sudan,Win,44,113,Lose
14729,2012-01-13,Tunisia,Côte d'Ivoire,Lose,59,16,Win
14730,2012-01-14,Angola,Sierra Leone,Win,83,60,Win
14731,2012-01-14,Zambia,Namibia,Draw,79,121,Lose
14732,2012-01-15,Oman,Congo DR,Draw,85,125,Lose


La exactitud de el ranking FIFA es de aporximadamente 22% y la del modelo elegido es de 53%, por lo que se creo un modelo con mayor exactitud; sin embargo no es demasiado confiable.